## CNN Train
Author: Dalyah Aljamal, 21 April 2018
#This script will train the ICVL dataset to perform hand pose estimation
#Accuracy and loss will be calculated

In [1]:
#read and load data_preprocessing file
using Knet, Plots, JLD, NBInclude
nbinclude("data_preprocessing .ipynb")

## Training version 1

In [2]:
#8 Samples for the training , 4 is minibatch size to make sure the model is working fine 
#next assignment I will Train the model on the whole dataset
Atype = gpu() >= 0 ? KnetArray{Float32} : Array{Float32}

rootpath="imgDraft/Training" 
arr1 = ["1","2"]
arr2 = ["123","321"]
trn_labelpath="imgDraft/Training/labels.txt"
tst_labelpath="imgDraft/Testing/seq1_labels.txt"

tst_rootpath="imgDraft/Testing" 
tst_arr1 = ["Seq1"]

trn_labels= read_labels(trn_labelpath) # (48, )
xtrn= read_img(rootpath,arr1,arr2, Atype);
ytrn= read_labels(trn_labelpath);
#ytrn= convert(Array{UInt8},ytrn)

xtst= read_tst_img(tst_rootpath,tst_arr1, Atype);
ytst= read_labels(tst_labelpath);
#ytst= convert(Array{UInt8},ytst)

BS=4;
dtrn = dataset_minibatching(BS, xtrn, ytrn, Atype);
dtst= dataset_minibatching(BS, xtst, ytst, Atype);

summary(xtrn)

128×128×1×1 Array{Float32,4}
128×128×1×1 Array{Float32,4}
128×128×1×1 Array{Float32,4}
128×128×1×1 Array{Float32,4}
128×128×1×1 Array{Float32,4}
128×128×1×1 Array{Float32,4}
128×128×1×1 Array{Float32,4}
128×128×1×1 Array{Float32,4}
Array{ColorTypes.Gray{FixedPointNumbers.Normed{UInt16,16}},2}
(240, 320)
128×128×1×1 Array{Float32,4}
Array{ColorTypes.Gray{FixedPointNumbers.Normed{UInt16,16}},2}
(240, 320)
128×128×1×1 Array{Float32,4}
Array{ColorTypes.Gray{FixedPointNumbers.Normed{UInt16,16}},2}
(240, 320)
128×128×1×1 Array{Float32,4}
Array{ColorTypes.Gray{FixedPointNumbers.Normed{UInt16,16}},2}
(240, 320)
128×128×1×1 Array{Float32,4}
Array{ColorTypes.Gray{FixedPointNumbers.Normed{UInt16,16}},2}
(240, 320)
128×128×1×1 Array{Float32,4}


"128×128×1×8 Array{Float32,4}"

In [3]:
(x,y)= first(dtrn)
map(summary, (x,y))


("128×128×1×4 Knet.KnetArray{Float32,4}", "48×4 Knet.KnetArray{Float32,2}")

In [ ]:
xtrn[1,:,:,:]

In [4]:
#Checking Diemtionalities

#Desired Dim for (xtrn, ytrn, xtst, ytst)
#("128×128×1×60000 Array{Float32,4}", "60000-element Array{UInt8,1}",
#"128×128×1×10000 Array{Float32,4}", "10000-element Array{UInt8,1}")

println(map(summary,(xtrn,ytrn,xtst,ytst)))
println("Num of MiniBatches")
println(length(dtrn))# Return number of MiniBatches size
println("ytrn")

println(size(ytrn))
println(typeof(ytrn))
println("xtst")
for x in xtst
    println(size(x))
    println(summary(x))
    println(typeof(x))
    break;
    #println(size(y)) 
end
println("dtst")
for (x, y) in dtst
    println(typeof(x)) 
    println(size(y)) 
end

println("xtrn")
for x in xtrn
    println(size(x))
    println(typeof(x))
    break;
    #println(size(y)) 
end

println("dtrn")
for (x, y) in dtrn
    println(typeof(x)) 
    println(size(y)) 
    break;
end

#to return Dimentionality of each Minibatch
println("Dim of first MiniBtch")
println(summary(dtrn))
println(summary(dtrn))
#(x, y) = first(dtrn);
#println(dtrn)
#println(xtrn)

#for (x, y) in dtrn
 #   
#end

("128×128×1×8 Array{Float32,4}", "48×8 Array{Float32,2}", "128×128×1×5 Array{Float32,4}", "48×5 Array{Float32,2}")
Num of MiniBatches
2
ytrn
(48, 8)
Array{Float32,2}
xtst
()
Float32
Float32
dtst
Knet.KnetArray{Float32,4}
(48, 4)
Knet.KnetArray{Float32,4}
(48, 1)
xtrn
()
Float32
dtrn
Knet.KnetArray{Float32,4}
(48, 4)
Dim of first MiniBtch
2-element Array{Any,1}
2-element Array{Any,1}


## CNN Model

1. Predict Function
2. Loss Function
3. Loss Gradient
4. Prameter Intialization
5. Trian Function


## Hyber Parameters

Batch size: 128

Momentum: 0.9

Weight decay: 0.001

Learning  rate: 0.01

Epoch number: 100

PCA Components: 30

## LOSS

In [21]:
#Loss Functions

# NLL Loss
#loss(w,x,ygold) = nll(predict(w,x),ygold)

#Mean Loss
#loss(w,x,y)= mean(abs2,y-predict(w,x))
function loss(w,x,y)
    #y= convert(Array{Float32,2},y)
    #y= convert(Atype,y)
    return mean(abs2,y-predict(w,x))
end

#Huber Loss
function huber_loss(w, x, y ; e=0.4)
    ypred=predict(w,x)
    N= size(y,1)
    E=e# ill(e, N)
    com=abs(y - ypred) .<= E
    
    out = true
    for i in 1:length(s)
        if  s[i] != true
            out=false
        end
    end
    if out
        lo= 0.5* (y- ypred).^2
    else
        lo=e* abs(y - ypred) - 0.5 * (E .^2)
        
    end
    return lo
end

#Gradient Loss
lossgradient = grad(loss);

## CNN Model Predict 

In [15]:
#Conv 8(5*5) , Max Pool (4*4), Conv 8(5*5), Max Pool (2*2), Conv 8(3*3), 2 FC (1024),droupout, PCA (30), FC(48)
function predict(w,x; pdrop=0.3)
    #win=[4,0,2]
    
    #n=length(w)-8
    #for i=1:2:n # we are jumping 2 steps since alwase we have w then b, then w  then b....
     #   x = pool(relu.(conv4(w[i],x) .+ w[i+1]); window=win[i]) 
    #end
    #i=n+1
    #x= relu.(conv4(w[i],x) .+ w[i+1])
    #for i=n+3:2:length(w)-2 # Fully connected layers
    #    x = relu.(w[i]*mat(x) .+ w[i+1])
    #end
    # Add PCA layer later
    #print(size( w[end-1]*mat(x) .+ w[end]  ))
    #return w[end-1]*mat(x) .+ w[end]
    #x= convert(Atype,x)
    #First Conv Layer
    x = pool(relu.(conv4(w[1],x) .+ w[2]); window=4) 
    
    #Second Conv Layer
    x = pool(relu.(conv4(w[3],x) .+ w[4]); window=2) 
    
    #Third Conv Layer
    x = relu.(conv4(w[5],x) .+ w[6])
    
    #FC 1
    x = relu.(w[7]*mat(x) .+ w[8])
    
    #FC 2
    x = relu.(w[9]* x .+ w[10])
    
    #Dropout layer
    x= dropout(x,pdrop);
    
    #PCA layer
    
    #OUTPUT
    #println(summary(w[11]* x .+ w[12]))
    return w[11]* x .+ w[12]
    
end

predict (generic function with 1 method)

## Parameter Intialization

In [70]:

#DimensionMismatch("((1024, 103968), (968, 120), (1024, 120))")

wcnn=map(Atype, [ 0.1*randn(5,5,1,8),  zeros(1,1,8,1), 
        0.1*randn(5,5,8,8),  zeros(1,1,8,1),
        0.1*randn(3,3,8,8),  zeros(1,1,8,1), #103968
        0.1*randn(1024,968),  zeros(1024,1),
        0.1*randn(1024,1024),  zeros(1024,1), # add PCA Layer parameters later
        0.1*randn(48,1024), zeros(48,1)]);

In [71]:
#TRAIN CNN Function

function train!(w, data, optims; lr=.01)
    for (x,y) in data
        #println("sqr error loss")
        #print(loss(w,x,y))
        #println("Huber Loss")
        #print(huber_loss(w,x,y))
        println("dw")
        @show (predict(w,x))
        dw = lossgradient(w, x, y)
        #println(dw = lossgradient(w, x, y))
        #println(size(dw))
        @show (map(vecnorm, dw))
        update!(w, dw, optims)
        #for i in 1:length(w) # Loop  over W and B
        #   #@time print(w[i] -= lr * dw[i])
        #    w[i] -= lr * dw[i]
        #end
    end
    
    return w
end

train! (generic function with 2 methods)

## TRAINING

In [72]:
#MAIN
optims = optimizers(wcnn, Adam)
@time cnnmodels = [ copy(train!(wcnn, dtrn, optims)) for epoch=1:100 ];


dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012051dd600, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[802.504, 497.618, 519.935, 376.696, 643.205, 792.068, 534.467, 309.728, 559.257, 150.664, 610.925, 67.8713]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012051e0600, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[3110.4, 1446.19, 2054.86, 1558.66, 1675.57, 2464.99, 562.264, 319.816, 560.679, 143.38, 634.319, 66.0653]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012051d7a00, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[6550.64, 3465.41, 4898.73, 3693.63, 3850.25, 5028.02, 745.811, 356.116, 756.933, 152.698, 909.36, 67.2649]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012051c3600, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[10200.8, 4943.35, 7970.89, 5608.29, 6246.29, 7378.64, 887.363, 372.715, 878.185, 143.749, 1240.11, 65.2325]

map(vecnorm, dw) = Float32[29413.3, 13930.3, 15739.3, 7243.85, 13875.4, 5743.32, 931.101, 84.9642, 1478.73, 19.7963, 4059.3, 7.20438]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012a2357a00, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[27417.6, 13617.3, 14804.0, 6991.08, 13552.4, 5576.6, 888.081, 80.2448, 1348.16, 17.846, 3667.43, 6.30404]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012a235ee00, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[36383.9, 17276.3, 19569.3, 8958.56, 17382.3, 7082.81, 1112.69, 100.236, 1548.9, 20.5171, 3782.78, 6.61846]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012a2366200, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[33989.5, 16587.1, 18371.8, 8567.58, 16698.6, 6838.89, 1064.39, 96.3281, 1445.86, 19.2733, 3535.31, 6.15072]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012a236d600, 768, 0, nothing), (48, 4))


map(vecnorm, dw) = Float32[8829.16, 4203.44, 4812.94, 2272.33, 4270.22, 1859.11, 335.542, 33.28, 668.868, 10.2764, 2259.86, 4.4336]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x000000134eda4000, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[5649.05, 2914.27, 3046.77, 1544.68, 2828.95, 1284.67, 297.374, 29.9557, 725.4, 11.2546, 2227.12, 4.37973]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x000000134eda6000, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[25724.0, 12158.7, 13982.9, 6603.12, 12212.6, 5392.31, 796.753, 80.1262, 1010.29, 15.6431, 2384.97, 4.70685]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x000000134eda8000, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[3502.32, 1773.36, 1911.05, 953.473, 1756.99, 795.405, 246.514, 24.9207, 720.254, 11.2511, 2478.83, 5.08912]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x000000134edaa000, 768, 0, nothing), (48, 4))
m

dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012051dd600, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[15086.7, 7171.59, 8242.77, 3975.4, 7130.54, 3302.1, 497.719, 53.3831, 720.568, 12.3879, 1900.63, 4.05892]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012051e0600, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[24575.6, 11936.2, 13439.5, 6606.35, 11873.5, 5540.8, 784.726, 84.771, 978.831, 16.8731, 2506.38, 5.98275]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012051d7a00, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[12939.1, 6036.16, 7054.06, 3424.59, 5911.12, 2842.45, 421.433, 46.4456, 648.843, 11.2855, 1830.46, 4.08144]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012051c3600, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[14405.5, 6905.98, 7838.15, 3813.08, 6918.82, 3195.9, 479.203, 51.1886, 704.749, 12.0066, 1975.15, 4.4382]
dw

map(vecnorm, dw) = Float32[4197.36, 2084.05, 2306.38, 1166.42, 2033.84, 992.488, 202.976, 22.948, 471.465, 8.64686, 1506.19, 3.53829]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012a2366200, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[13167.7, 6208.46, 7213.44, 3551.6, 6110.34, 3001.33, 420.162, 48.2783, 585.513, 10.8941, 1596.0, 3.94672]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012a236d600, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[1654.91, 723.002, 919.752, 420.409, 799.868, 366.847, 173.993, 19.4152, 474.918, 8.68602, 1545.31, 3.64056]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012a2374a00, 768, 0, nothing), (48, 4))
map(vecnorm, dw) = Float32[13064.0, 6231.94, 7177.03, 3501.48, 6228.97, 2952.03, 434.48, 48.0687, 604.242, 11.0469, 1613.82, 3.77498]
dw
predict(w, x) = Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x00000012a237be00, 768, 0, nothing), (48, 4))


In [77]:
#CNN Loss
#@time trncnnloss = [mean(abs2,ytrn-predict(w,xtrn)) for w in cnnmodels ]; 
#@time tstcnnloss = [ mean(abs2,ytst-predict(w,xtst)) for w in cnnmodels ]; 

trncnnloss=0
tstcnnloss =0
c=1
trncnnlosses= []
tstcnnlosses= []
@time for w in cnnmodels
    w= map(Atype, w)
    for (x,y) in dtrn
        #trncnnloss =+ mean(abs2,y-predict(wcnn,x))
        trncnnloss += loss(w,x,y)
        c+=1
    end
    push!(trncnnlosses, trncnnloss/c) #trncnnloss/c
end
    
for w in cnnmodels    
#Testing Set   
    for (x,y) in dtst
        tstcnnloss += loss(w,x,y)
    end
    push!(tstcnnlosses, tstcnnloss/c)
end

print(trncnnlosses)
print(tstcnnloss)

  0.298999 seconds (82.51 k allocations: 3.675 MiB)
Any[378.988, 519.037, 561.925, 554.114, 574.79, 576.486, 584.713, 593.064, 611.158, 607.367, 604.858, 605.494, 609.044, 603.849, 604.39, 603.811, 609.072, 612.083, 613.007, 609.43, 620.667, 617.886, 613.876, 622.801, 619.667, 626.117, 624.857, 624.554, 627.138, 629.612, 633.573, 631.334, 628.981, 626.664, 625.766, 627.749, 629.717, 631.548, 634.805, 636.425, 634.198, 634.454, 633.927, 634.004, 632.711, 631.888, 631.458, 633.641, 634.368, 632.473, 633.139, 633.56, 634.404, 635.961, 638.436, 635.932, 634.126, 634.485, 635.219, 633.556, 636.5, 637.346, 639.222, 641.495, 641.361, 640.826, 642.619, 642.929, 641.41, 642.735, 642.732, 641.58, 641.167, 640.058, 640.386, 640.204, 639.901, 641.102, 641.626, 640.023, 639.672, 640.29, 639.341, 640.94, 641.172, 640.682, 639.155, 640.025, 641.699, 641.081, 639.595, 639.88, 637.979, 637.839, 637.092, 636.547, 636.482, 637.069, 636.039, 636.092]214345.86

In [78]:
#Plotting Loss

plot( [trncnnlosses],ylim=(.000,.010),labels=[ :trncnnlosses ],xlabel="Epochs",ylabel="Loss") 

In [ ]:
##Accuracy

#trnAcc= accuracy(wcnn,dtrn,predict)
#tstAcc=accuracy(wcnn,dtst,predict)

#trnAcc=0
#tstAcc =0
#c=1

#for (x,y) in dtrn
   # y= convert(Array{Float32,2},y)
   # x= convert(Array{Float32,2},x)
   # trnAcc =+ accuracy(predict(wcnn,x),y)
    #c+=1
#end

#for (x,y) in dtst
#    y= convert(Array{Float32,2},y)
#    tstAcc = + accuracy(predict(wcnn,x),y) 
#end

#print(trnAcc/c)
#print(tstAcc/c)

#print(trnAcc)

## Training Version 2

In [ ]:
@doc minibatch